From http://www-inst.eecs.berkeley.edu/~cs61a/sp12/hw/hw2.html 

Q1. The summation function from lecture is only the simplest of a vast number of similar abstractions that can be captured as higher-order functions. Write a similar product function that returns the product of the values of a function for n natural number arguments. Show how to define the factorial function in terms of product:

In [6]:
def product(n, term):
    """Return the product of the first n terms in the sequence formed
    by applying term to the integers 1, ..., n.

    term -- a function that takes one argument
    """
    if n == 1:
        return n
    return n * product(term(n-1), term)

In [11]:
def factorial(n):
    """
    Returns the factorial of n.
    """
    return product(n, lambda x: x)

In [14]:
def test_factorial():
    assert factorial(4) == 24
    assert factorial(5) == 120
test_factorial()

Q2. Show that both summation and product are instances of a more general function, called accumulate.

Accumulate takes as arguments the same arguments term and n as summation and product, together with a combiner function (of two arguments) that specifies how the accumulation of the preceding terms is to be combined with each value returned by term, and a start value that specifies what base value to use to start the accumulation. Implement accumulate and show how summation and product can both be defined as simple calls to accumulate.

In [29]:
def accumulate(combiner, start, n, term):
    """Return the result of combining the first n terms in a sequence."""
    if n == start:
        return term(n)
    return combiner(n, accumulate(combiner, start, term(n-1), term))

In [30]:
def product_acc(n, term):
    """
    Redefinition of the 'product' function earlier to use accumulate
    """
    return accumulate(lambda a, b: a * b, 1, n, term)

def factorial_acc(n):
    """
    Returns the factorial of n.
    """
    return product_acc(n, lambda x: x)

def test_factorial_acc():
    assert factorial_acc(4) == 24
    assert factorial_acc(5) == 120
test_factorial_acc()

In [32]:
def summation_acc(n, term):
    """
    Redefinition of summation to use accumulate
    """
    return accumulate(lambda a, b: a + b, 1, n, term)

def simple_sum(n):
    return summation_acc(n, lambda x: x)

def test_summation_acc():
    assert simple_sum(5) == 15
    assert simple_sum(1) == 1
test_summation_acc()

Q3. Define a function double that takes a function of one argument as an argument and returns a function that applies the original function twice. For example, if successor is a function that returns 1 more than its argument, then double(inc) should be a function that returns two more:

In [36]:
def double(f):
    """Return a function that applies f twice."""
    return lambda x: f(f(x))

def test_double():
    assert double(simple_sum)(5) == simple_sum(simple_sum(5))
test_double()

Q4. If f is a numerical function and n is a positive integer, then we can form the nth repeated application of f, which is defined to be the function whose value at x is f(f(...(f(x))...)). For example, if f adds 1 to its argument, then the nth repeated application of f adds n. Write a function that takes as inputs a function f and a positive integer n and returns the function that computes the nth repeated application of f:

In [45]:
def repeated(f, n):
    """Return the function that computes the nth application of f."""
    if n == 1:
        return f
    def compose(a, b):
        return lambda x: a(b(x))
    return lambda x:  f(repeated(f, n-1)(x))

In [46]:
def test_repeated():
    assert repeated(lambda x: x**2, 2)(5) == 625
test_repeated()

Extra for Experts: Q5. The logician Alonzo Church invented a system of representing non-negative integers entirely using functions. Here are the definitions of 0, and a function that returns 1 more than its argument:

In [47]:
def zero(f):
    return lambda x: x

def successor(n):
    return lambda f: lambda x: f(n(f)(x))

This representation is known as Church numerals. Define one and two directly, which are also functions. To get started, apply successor to zero. Then, give a direct definition of the add function (not in terms of repeated application of successor) over Church numerals. Finally, implement a function church_to_int that converts a church numeral argument to a regular Python int.

It is easy to find answers to this question on the Internet. Try to solve it on your own before looking it up!

In [58]:
def one(f):
    return f # By substituting and simplifying n=zero in sucessor

In [59]:
def two(f):
    return lambda x: f(f(x))

In [172]:
def add(f, g):
    return lambda y: lambda x: f(y)(g(y)(x)) # I still need to unpack how this works

So the to integer setup was what I first wrote, and I somehow wrote up the solution and then was not sure how it worked almost immediately. This feels like one of those things where the answer is always something that's just within grasp but not quite...

Ok, so I think a Church Number n(f(x)) is defined as the number of times you are recursively applying an arbitrary function f n times starting at x.

So if we apply (x + 1) n times starting at 0, we'll get n. 

For say, '3', it expands to:
    0 + 1 = 0
    1 + 1 = 2
    2 + 1 = 3
    
Addition works by saying - once you've applied it f times, apply it again g times! Hence `f(y)(g(y)(x))`

Let's write that out for f = 3, g = 2, and the function is x + 1 again and starts at 0

    g(x+1)(0)
    0 + 1 = 1
    1 + 1 = 2
    
Then we pass that as the *starting point* for f

    f(x+1)(2)
    2 + 1 = 3
    3 + 1 = 4
    4 + 1 = 5
   

In [187]:
def church_to_int(n):
    return n(lambda x: x + 1)(0) # I have no memory of how I hit this as an answer...
church_to_int(successor(two))

3

In [188]:
church_to_int(add(successor(two), zero))

3